<a href="https://colab.research.google.com/github/Maruf346/AI-ML-with-python/blob/main/Thesis_MADDPG_Dataset_for_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import pandas as pd
import random

# --- 1. CONFIGURATION ---
NUM_ROWS = 5000
NUM_DEVICES = 30
OUTPUT_FILENAME = 'PRiFARED_maddpg_dataset_5000.csv'

# Define the precise ranges for min-max scaling later
RANGES = {
    'C_cpu': (10, 100),
    'C_data': (5, 50),
    'T_max': (100, 500),
    'Rn': (0, 9), # Integer range
    'Battery Level (%)': (1, 100),
    'Q_k': (0, 50),
    'F_max': (1, 5),
    'B_max': (5, 30),
    'P_max': (1, 6),
    'x_hat_alloc': (0, 1), # Allocations are fractions [0, 1]
    # Placeholders for results
    'Latency': (50, 600),
    'Energy': (0.5, 12.0)
}

# Define categorical mappings
TASK_CLASSES = {0: '0 (Normal)', 1: '1 (Medium)', 2: '2 (Urgent)'}
USER_CONTRACTS = {0: '0 (Normal)', 1: '1 (Premium)'}

# --- 2. HELPER FUNCTIONS ---

def generate_continuous(min_val, max_val, num_samples):
    """Generates random continuous values within a specified range."""
    return np.random.uniform(min_val, max_val, num_samples)

def generate_integer(min_val, max_val, num_samples):
    """Generates random integer values within a specified range."""
    return np.random.randint(min_val, max_val + 1, num_samples)

def generate_vector(min_val, max_val, num_samples, vector_size):
    """Generates vectors (lists of floats) for device parameters."""
    data = []
    for _ in range(num_samples):
        # Generate random values for the vector
        vector = np.random.uniform(min_val, max_val, vector_size)
        # Store as a string representation to match the original markdown format
        data.append('$$' + ', '.join(f'{x:.2f}' for x in vector) + '$$')
    return data

def generate_allocation_vector(num_samples, vector_size):
    """Generates a normalized allocation vector (sums to 1.0)."""
    data = []
    for _ in range(num_samples):
        # Generate raw random values
        raw_allocs = np.random.uniform(0.01, 1.0, vector_size)
        # Normalize so they sum to 1.0 (to respect resource limits)
        normalized_allocs = raw_allocs / np.sum(raw_allocs)
        # Store as a string
        data.append('$$' + ', '.join(f'{x:.3f}' for x in normalized_allocs) + '$$')
    return data

# --- 3. DATA GENERATION ---

data = pd.DataFrame()

# 1. Task Feature Inputs
data['Task ID'] = np.arange(1, NUM_ROWS + 1)
data['C_cpu (GC) [10-100]'] = generate_continuous(*RANGES['C_cpu'], NUM_ROWS)
data['C_data (Mbits) [5-50]'] = generate_continuous(*RANGES['C_data'], NUM_ROWS)
data['T_max (ms) [100-500]'] = generate_continuous(*RANGES['T_max'], NUM_ROWS)
data['Rn (Retry) [0-9]'] = generate_integer(*RANGES['Rn'], NUM_ROWS)
data['Battery Level (%) [1-100]'] = generate_continuous(*RANGES['Battery Level (%)'], NUM_ROWS)

# 2. Categorical Inputs (using mappings)
task_classes_raw = generate_integer(0, 2, NUM_ROWS)
user_contracts_raw = generate_integer(0, 1, NUM_ROWS)
data['Task_Service_Class [0/1/2]'] = [TASK_CLASSES[c] for c in task_classes_raw]
data['User_Contract_Type [0/1]'] = [USER_CONTRACTS[c] for c in user_contracts_raw]

# 3. Environment Vector Inputs (4 devices)
data['Q_k (Queue) [0-50]'] = generate_vector(*RANGES['Q_k'], NUM_ROWS, NUM_DEVICES)
data['F_max (GHz) [1-5]'] = generate_vector(*RANGES['F_max'], NUM_ROWS, NUM_DEVICES)
data['B_max (MHz) [5-30]'] = generate_vector(*RANGES['B_max'], NUM_ROWS, NUM_DEVICES)
data['P_max (W) [1-6]'] = generate_vector(*RANGES['P_max'], NUM_ROWS, NUM_DEVICES)

# 4. Action Placeholder Outputs (Normalized allocations)
data['x_hat_cpu (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_bw (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_pow (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)

# 5. Result Placeholder Outputs (These are synthetic results for a dataset)
data['Latency (ms) [Result]'] = generate_continuous(*RANGES['Latency'], NUM_ROWS)
data['Energy (J) [Result]'] = generate_continuous(*RANGES['Energy'], NUM_ROWS)
# Success is 1 if Latency is less than T_max, 0 otherwise (approximated for this placeholder data)
data['Success (0/1) [Result]'] = (data['Latency (ms) [Result]'] < data['T_max (ms) [100-500]']).astype(int)

# --- 4. OUTPUT AND DISPLAY ---

# Save the dataset to a CSV file (typical for Colab or training environments)
data.to_csv(OUTPUT_FILENAME, index=False)

print(f"Successfully generated {NUM_ROWS} rows of data.")
print(f"Data saved to {OUTPUT_FILENAME}")
print("\nFirst 5 rows:")
print(data.head())
print(f"\nDataFrame shape: {data.shape}")


Successfully generated 5000 rows of data.
Data saved to PRiFARED_maddpg_dataset_5000.csv

First 5 rows:
   Task ID  C_cpu (GC) [10-100]  C_data (Mbits) [5-50]  T_max (ms) [100-500]  \
0        1            39.275663              49.640683            205.478771   
1        2            13.012940              46.492622            363.515507   
2        3            35.587507              42.658673            156.164179   
3        4            30.943167              34.185469            328.037306   
4        5            14.149368              47.703262            430.910190   

   Rn (Retry) [0-9]  Battery Level (%) [1-100] Task_Service_Class [0/1/2]  \
0                 7                  31.110672                 2 (Urgent)   
1                 9                  25.885311                 2 (Urgent)   
2                 8                  72.623973                 1 (Medium)   
3                 4                  83.073109                 1 (Medium)   
4                 5           

In [7]:
display(data)

,Task ID,C_cpu (GC) [10-100],C_data (Mbits) [5-50],T_max (ms) [100-500],Rn (Retry) [0-9],Battery Level (%) [1-100],Task_Service_Class [0/1/2],User_Contract_Type [0/1],Q_k (Queue) [0-50],F_max (GHz) [1-5],B_max (MHz) [5-30],P_max (W) [1-6],x_hat_cpu (Alloc) [0-1],x_hat_bw (Alloc) [0-1],x_hat_pow (Alloc) [0-1],Latency (ms) [Result],Energy (J) [Result],Success (0/1) [Result]
0,1,39.275663,49.640683,205.478771,7,31.110672,2 (Urgent),1 (Premium),"$$22.88, 9.15, 5.76, 7.87, 37.39, 34.22, 49.37...","$$2.54, 4.76, 3.76, 3.48, 2.36, 3.93, 1.46, 1....","$$10.38, 11.77, 23.09, 23.75, 18.20, 19.79, 18...","$$5.35, 4.63, 2.54, 1.79, 1.10, 5.25, 5.81, 4....","$$0.046, 0.032, 0.018, 0.010, 0.055, 0.065, 0....","$$0.054, 0.021, 0.025, 0.019, 0.024, 0.009, 0....","$$0.027, 0.043, 0.019, 0.048, 0.039, 0.027, 0....",143.215906,5.834788,1
1,2,13.012940,46.492622,363.515507,9,25.885311,2 (Urgent),1 (Premium),"$$27.43, 6.48, 37.24, 3.56, 19.47, 32.79, 10.8...","$$2.57, 2.45, 2.32, 2.58, 4.91, 2.56, 4.38, 1....","$$18.39, 9.61, 19.05, 17.75, 12.94, 27.63, 9.9...","$$3.24, 1.34, 4.59, 3.31, 3.98, 3.16, 5.20, 4....","$$0.033, 0.030, 0.030, 0.047, 0.025, 0.053, 0....","$$0.035, 0.007, 0.048, 0.010, 0.052, 0.052, 0....","$$0.057, 0.018, 0.031, 0.009, 0.059, 0.052, 0....",370.209122,5.795869,0
2,3,35.587507,42.658673,156.164179,8,72.623973,1 (Medium),0 (Normal),"$$19.88, 5.86, 19.76, 6.16, 46.75, 17.57, 22.7...","$$3.00, 2.86, 3.56, 1.17, 4.35, 2.60, 1.98, 2....","$$28.85, 19.29, 23.56, 16.18, 17.11, 14.72, 13...","$$3.30, 1.40, 5.76, 2.51, 5.37, 1.16, 1.24, 3....","$$0.053, 0.046, 0.049, 0.043, 0.004, 0.049, 0....","$$0.050, 0.003, 0.006, 0.061, 0.012, 0.036, 0....","$$0.058, 0.027, 0.040, 0.043, 0.026, 0.019, 0....",63.336050,11.625284,1
3,4,30.943167,34.185469,328.037306,4,83.073109,1 (Medium),1 (Premium),"$$16.85, 31.00, 32.36, 18.65, 30.37, 37.68, 17...","$$2.89, 2.85, 1.21, 2.98, 4.46, 3.14, 3.23, 4....","$$5.94, 9.19, 21.60, 28.82, 19.37, 16.39, 25.7...","$$5.58, 1.04, 3.63, 3.78, 3.20, 3.04, 2.90, 2....","$$0.058, 0.019, 0.016, 0.030, 0.004, 0.046, 0....","$$0.052, 0.024, 0.010, 0.003, 0.020, 0.017, 0....","$$0.030, 0.053, 0.011, 0.029, 0.042, 0.048, 0....",155.619569,11.701880,1
4,5,14.149368,47.703262,430.910190,5,28.540163,1 (Medium),1 (Premium),"$$12.97, 33.36, 27.17, 34.69, 18.08, 43.75, 2....","$$1.95, 3.50, 3.52, 1.81, 4.09, 3.07, 1.61, 3....","$$6.14, 21.94, 20.44, 24.47, 7.44, 5.10, 29.22...","$$2.50, 3.71, 2.18, 5.67, 4.07, 3.88, 2.88, 5....","$$0.055, 0.027, 0.024, 0.058, 0.019, 0.047, 0....","$$0.016, 0.060, 0.030, 0.005, 0.049, 0.038, 0....","$$0.028, 0.028, 0.010, 0.012, 0.031, 0.030, 0....",339.190616,1.344184,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,65.152834,48.211228,449.400897,2,86.803837,1 (Medium),1 (Premium),"$$18.84, 22.42, 18.55, 12.98, 1.28, 23.96, 17....","$$2.31, 4.10, 4.95, 4.50, 4.80, 4.31, 1.83, 4....","$$28.72, 20.41, 9.61, 18.18, 9.82, 20.04, 11.0...","$$5.65, 3.46, 3.82, 4.17, 2.12, 1.97, 4.46, 5....","$$0.057, 0.053, 0.042, 0.056, 0.041, 0.041, 0....","$$0.063, 0.014, 0.009, 0.016, 0.028, 0.034, 0....","$$0.003, 0.065, 0.006, 0.014, 0.003, 0.061, 0....",118.686589,9.549441,1
4996,4997,53.794069,10.046502,112.982008,7,15.919914,0 (Normal),1 (Premium),"$$23.80, 28.30, 15.70, 13.89, 20.57, 21.93, 47...","$$2.05, 4.09, 1.36, 4.50, 2.98, 3.47, 2.68, 4....","$$8.82, 24.13, 29.74, 18.11, 10.16, 25.90, 28....","$$2.26, 1.36, 3.11, 3.20, 3.05, 3.35, 3.32, 3....","$$0.067, 0.037, 0.017, 0.059, 0.008, 0.001, 0....","$$0.049, 0.006, 0.049, 0.055, 0.043, 0.053, 0....","$$0.063, 0.057, 0.032, 0.059, 0.059, 0.058, 0....",173.182062,11.287197,0
4997,4998,83.717972,38.534638,271.760648,5,81.708788,2 (Urgent),0 (Normal),"$$13.49, 24.01, 34.43, 14.28, 3.31, 8.14, 44.5...","$$2.86, 3.29, 4.86, 1.62, 4.56, 2.29, 3.30, 1....","$$23.47, 27.59, 18.37, 14.10, 26.70, 19.39, 15...","$$4.76, 1.25, 5.78, 5.30, 3.89, 4.85, 4.83, 5....","$$0.020, 0.045, 0.046, 0.017, 0.029, 0.034, 0....","$$0.048,